In [120]:
import pandas as pd
import numpy as np
import cPickle as pickle

In [121]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

# Se carga el set de entrenamiento

In [163]:
prop = pickle.load(open("setEntrenamiento.p","rb"))

In [164]:
#cortamos por precios
prop = prop[(prop['price_usd_per_m2']<9062)]
prop.reset_index(inplace=True)

p = prop[['property_type','place_name','surface_total_in_m2',\
          'price_usd_per_m2','price_aprox_usd']]

### Se le asigna un valor numerico al tipo de propiedad

In [165]:
def asignarPT( p ):
    if p == 'apartment':
        return 3
    if p == 'house':
        return 2.25
    if p == 'store':
        return 6.5
    return 2.5

In [166]:
p['property_type'] = p.apply(lambda x: asignarPT(x[0]), axis = 1)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### Se le asigna un valor a los barrios segun el analisis de grupos hecho en el tp1

In [167]:
def asignarPV(name, dic):
    return dic[name]   

In [168]:
hoodGroups = p[["place_name", "price_usd_per_m2"]]
hoodGroups = hoodGroups.groupby("place_name").agg([np.mean]).reset_index()
hoodGroups = hoodGroups.sort_values(by=("price_usd_per_m2", "mean"), ascending=False)

In [169]:
HGdics = {}
precio_ant = 0
val_ant = 0
for row in hoodGroups.iterrows():
    name = row[1][0]
    price = row[1][1]
    val = val_ant + (precio_ant/price)
    HGdics[name] = val
    precio_ant = price
    val_ant = val

In [170]:
p["place_value"] = p.apply(lambda x: asignarPV(x[1], HGdics), axis=1)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [171]:
pEnt = p[['property_type','surface_total_in_m2',\
    'place_value', 'price_aprox_usd']]
pEnt.dropna(inplace=True)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# Se carga el set de Prueba

In [172]:
proper = pickle.load(open("setPrueba.p","rb"))

In [173]:
#se formatea lat y lon
p = proper[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'latlon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte']]

p['lat'] = p.apply(lambda row: row[4][0] if(np.all(pd.notnull(row[5]))) else row[5], axis=1)

p['lon'] = p.apply(lambda row: row[4][1] if(np.all(pd.notnull(row[5]))) else row[5], axis=1)

p = p[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte']]

p['property_type'] = p.apply(lambda x: asignarPT(x[0]), axis = 1)

def asignarPVsegunEnt(name):
    if(HGdics.has_key(name)):
        return HGdics[name]
    return np.nan

#hay que armar los hoodGropus segun los resultados del set de Entrenamiento
p["place_value"] = p.apply(lambda x: asignarPVsegunEnt(x[1]), axis=1)

pPru = p[['property_type','surface_total_in_m2',\
        'lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'place_value', 'place_name']]

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


### Prediccion del precio

In [180]:
#definir el modelo a probar
model = GradientBoostingRegressor(loss = 'ls', n_estimators = 5000, max_depth = 5, learning_rate = 0.241)

In [181]:
#fit model on training dataset
model.fit(pEnt[['property_type','surface_total_in_m2','place_value']], pEnt['price_aprox_usd'])

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.241, loss='ls', max_depth=5,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=5000,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [182]:
#prediccion 
pred = mean_squared_error(pEnt['price_aprox_usd'], model.predict(pEnt[['property_type','surface_total_in_m2',\
    'place_value']]))
print ('RMSE_Price', pred)

('RMSE_Price', 6909913267.7700148)


In [185]:
from sklearn.preprocessing import Imputer
#hay que llenar el resto de los valores nan

# Create our imputer to replace missing values with the mean e.g.
imp = Imputer(missing_values=np.nan, strategy='most_frequent', axis=1)
imp = imp.fit(pEnt)

pPru_imp = imp.transform(pPru[['property_type','surface_total_in_m2',
        'place_value']])

In [186]:
pPru['price_usd'] = model.predict(pPru_imp)
result = pd.DataFrame()
result["id"] = proper["id"]
result["price_usd"] = pPru['price_usd']
result.head()

,id,price_usd
0,2316505,168594.991479
1,2320880,198785.527147
2,2321742,216718.430634
3,2276796,56171.748787
4,2330364,67038.237455


In [187]:
result.to_csv(index=False, path_or_buf="resultsFinal2PhiAlpha.csv")